## Files inside Logs

## Time_series_modelling_+_feature_extraction_+_transformer.ipynb

The transformer method is still a work in progress. It was successful in compressing the entire dataset by 100%. So each raw time series data went from 750k+ rows to 75k+ rows. However, the only issue is that this decreases the accuracy, as I have not found a classifer that works well in interpreting the features that the transformer condensed.

We also attempt to use Deep Embedded Clustering (called DEC) to cluster the compressed time series data into 5 clusters WITHOUT SUPERVISION. The 5 clusters were decided by the model, yet cross-checking this with groundtruth, we get mixed results. Sometimes, two classes of trees belonging to one cluster.

For instance, the model classifies a set of points under cluster 1, yet cross-checking these points with groundtruth, we get larch_CN and shrubland both belonging to cluster 1. The results from DEC are not satisfactory, indicating that by running a transformer to compress the data, we are decreasing the "distinctiveness" of each tree species class with one another.

That is my critique of using transformers to encode and compress time series data. Regardless, if you want to see the process of going through and testing everything, go to the original time_series_modelling ipynb file in logs directory


## Groundtruth_analysis.ipynb

This piece of code focuses mainly on LSTM approach (semi-supervised). The code there works and most of the code here in main workflow is inspired from groundtruth analysis code located in logs.

## What's inside this code?
### This is the main pipeline that works. Components:
1. GEE time series sampler code -- exports it into Google Drive, check task manager
2. CSV cleaner -- remove the .geo coordinate and run regex to extract lat long, also implement a CSV summarizer.
3. LSTM model; classifed on unseen data

## (1) Time-Series Sampler (written in Javascript, execute in GEE)

1. Define a study area polygon, compute its centroid, and adjust dimensions (m to degrees) for grid alignment.
2. Generate a grid of 256×256 patches by converting patch size (2560 m) to degrees.
3. Filter and process Sentinel-2 images (cloud masking, NDVI/EVI/NDWI computation, band selection).
4. For each patch and month, sample the earliest image's pixels and export as CSV into your Google Drive. Copy paste the code into your Google Earth Engine.


In [ ]:
// ======================================================
// GEE Script: Export One CSV Per 256x256 Patch Over an Adjusted Study Area (using lat/lon arithmetic)
// ======================================================

// ----- 1. Define the Original Study Area Polygon (in lat/lon) -----
var origPolygon = ee.Geometry.Polygon([
  [
    [107.57481296526105, 33.62952626245739],
    [107.67265995012433, 33.62952626245739],
    [107.67265995012433, 33.786036563473694],
    [107.57481296526105, 33.786036563473694],
    [107.57481296526105, 33.62952626245739]
  ]
]);

// Compute the centroid of the original polygon.
var centroid = origPolygon.centroid();
var centerCoords = centroid.coordinates();
var centerLon = ee.Number(centerCoords.get(0));
var centerLat = ee.Number(centerCoords.get(1));
print("Centroid (lon, lat):", centerLon, centerLat);

// ----- 2. Define the New Study Area Using Approximate Meter-to-Degree Conversions -----
// Desired new dimensions (in meters) to allow an integer number of 256×256 patches.
// For 28 patches we choose: width = 10,240 m, height = 17,920 m.
var desiredWidthMeters = 10240;
var desiredHeightMeters = 17920;

// Conversion factors at the centroid (approximate)
var metersPerDegLat = 111320;  // roughly
var cosLat = centerLat.multiply(Math.PI).divide(180).cos();
var metersPerDegLon = ee.Number(111320).multiply(cosLat);

// Compute new dimensions in degrees.
var newWidthDeg = ee.Number(desiredWidthMeters).divide(metersPerDegLon);
var newHeightDeg = ee.Number(desiredHeightMeters).divide(metersPerDegLat);
print("New study area dimensions (degrees):", newWidthDeg, "x", newHeightDeg);

// Define the new study area (centered on the centroid).
var halfWidthDeg = newWidthDeg.divide(2);
var halfHeightDeg = newHeightDeg.divide(2);
var newLonMin = centerLon.subtract(halfWidthDeg);
var newLonMax = centerLon.add(halfWidthDeg);
var newLatMin = centerLat.subtract(halfHeightDeg);
var newLatMax = centerLat.add(halfHeightDeg);
var newStudyArea = ee.Geometry.Rectangle([newLonMin, newLatMin, newLonMax, newLatMax]);
Map.centerObject(newStudyArea, 10);
Map.addLayer(newStudyArea, {color: 'red'}, 'Adjusted Study Area');

// ----- 3. Generate a Grid of 256x256 Patches in Degrees -----
// At 10 m resolution, a 256×256 patch covers 256*10 = 2560 m on a side.
// Convert 2560 m to degrees using the same conversion factors.
var patchWidthDeg = ee.Number(2560).divide(metersPerDegLon);
var patchHeightDeg = ee.Number(2560).divide(metersPerDegLat);
print("Patch size (degrees):", patchWidthDeg, "x", patchHeightDeg);

// Compute number of patches horizontally and vertically.
var numCols = newWidthDeg.divide(patchWidthDeg).floor();
var numRows = newHeightDeg.divide(patchHeightDeg).floor();
print("Number of patches (cols x rows):", numCols.getInfo(), "x", numRows.getInfo(),
      "=", numCols.multiply(numRows).getInfo(), "patches.");

// Create grid features.
var gridFeatures = [];
var nCols = numCols.getInfo();
var nRows = numRows.getInfo();
for (var i = 0; i < nCols; i++) {
  for (var j = 0; j < nRows; j++) {
    var lon0 = newLonMin.add(ee.Number(i).multiply(patchWidthDeg));
    var lat0 = newLatMin.add(ee.Number(j).multiply(patchHeightDeg));
    var lon1 = lon0.add(patchWidthDeg);
    var lat1 = lat0.add(patchHeightDeg);
    var patchGeom = ee.Geometry.Rectangle([lon0, lat0, lon1, lat1]);
    var patchFeature = ee.Feature(patchGeom, { id: 'patch_' + i + '_' + j });
    gridFeatures.push(patchFeature);
  }
}
var gridFC = ee.FeatureCollection(gridFeatures);
print("Total patches in grid:", gridFC.size());
Map.addLayer(gridFC, {color: 'blue'}, "Grid Patches");

// ----- 4. Define Date Range and Sentinel-2 Image Collection -----
var startDate = '2022-01-01';
var endDate   = '2022-12-31';
var s2Collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
  .filterDate(startDate, endDate)
  .filterBounds(newStudyArea)
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20));

// ----- 5. Define Functions for Image Processing (Without DEM) -----
function maskClouds(image) {
  var qa = image.select('QA60');
  var cloudBitMask = 1 << 10;
  var cirrusBitMask = 1 << 11;
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0)
              .and(qa.bitwiseAnd(cirrusBitMask).eq(0));
  return image.updateMask(mask);
}

function addIndices(image) {
  var ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI');
  var evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
      'NIR': image.select('B8'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')
    }
  ).rename('EVI');
  var ndwi = image.expression(
    '(GREEN - NIR) / (GREEN + NIR)',
    {
      'GREEN': image.select('B3'),
      'NIR': image.select('B8')
    }
  ).rename('NDWI');
  return image.addBands([ndvi, evi, ndwi]);
}

function selectBands(image) {
  var originalBands = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12','AOT','WVP'];
  var rgbBands = ['TCI_R','TCI_G','TCI_B'];
  var indexBands = ['NDVI','EVI','NDWI'];
  return image.select(originalBands.concat(rgbBands).concat(indexBands));
}

var s2Processed = s2Collection
  .map(maskClouds)
  .map(addIndices)
  .map(selectBands)
  .map(function(img) { return img.unmask(-999); });

// ----- 6. For Each Patch and Each Month, Sample Pixels -----
// For each patch and for each month (1–12), select the earliest image and sample every pixel.
var months = ee.List.sequence(1, 12);
var patchList = gridFC.toList(gridFC.size());
var numPatches = patchList.size().getInfo();
print("Number of patches to process:", numPatches);

for (var i = 0; i < numPatches; i++) {
  var patch = ee.Feature(patchList.get(i));
  var patchId = patch.get('id');
  print("Processing patch:", patchId);

  var monthlySamples = months.map(function(m) {
    var monthlyImages = s2Processed
      .filter(ee.Filter.calendarRange(m, m, 'month'))
      .filterBounds(patch.geometry());
    var sorted = monthlyImages.sort('system:time_start');
    var firstImage = ee.Image(sorted.first());
    var samples = ee.Algorithms.If(
      sorted.size().gt(0),
      firstImage.sample({
        region: patch.geometry(),
        scale: 10,
        projection: 'EPSG:4326',
        geometries: true
      }).map(function(feat) {
        return feat.set({
          'image_date': firstImage.date().format('YYYY-MM-dd'),
          'patch_id': patchId
        });
      }),
      ee.FeatureCollection([])
    );
    return ee.FeatureCollection(samples);
  });

  var patchSamples = ee.FeatureCollection(monthlySamples).flatten();
  print("Patch", patchId, "samples count:", patchSamples.size());

  Export.table.toDrive({
    collection: patchSamples,
    description: 'LarchCN_256x256_2022_monthly_' + patchId.getInfo(),
    fileFormat: 'CSV',
    folder: '2025-02-09_larch'
  });
}

print("Script executed. Check the Tasks tab for exports.");


## (2) CSV summarizer, cleaner, checker

When you get the CSVs, there is a .geo column. Run this code in order to use regex to extract lat long coordinates from .geo column. We also drop other unused columns in the process, and then summarize the CSV.

#### GEE time series cleaner

In [ ]:
import pandas as pd
import re
import os
import glob
import csv
from tqdm import tqdm

# -------------------------------------------------------------------
# CSV Cleaner Function (Do Not Modify)
# -------------------------------------------------------------------
def clean_csv(input_path):
    # Read the CSV file
    df = pd.read_csv(input_path)

    # Extract latitude and longitude from .geo column using regex
    def extract_coordinates(geo_str):
        match = re.search(r'\[([-+]?\d*\.\d+),\s*([-+]?\d*\.\d+)\]', geo_str)
        if match:
            return float(match.group(1)), float(match.group(2))
        return None, None

    df['longitude'], df['latitude'] = zip(*df['.geo'].apply(extract_coordinates))

    # Drop unnecessary columns
    df = df.drop(columns=['system:index', 'month', '.geo'])

    # Ensure all feature columns are numeric
    feature_cols = [
        "AOT", "B1", "B11", "B12", "B2", "B3", "B4", "B5",
        "B6", "B7", "B8", "B8A", "B9", "EVI", "NDVI", "NDWI",
        "TCI_B", "TCI_G", "TCI_R", "WVP"
    ]
    for col in feature_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Handle missing values (forward fill)
    df[feature_cols] = df[feature_cols].ffill()

    # Save the cleaned CSV
    output_path = os.path.splitext(input_path)[0] + "_cleaned.csv"
    df.to_csv(output_path, index=False)
    print(f"Cleaned CSV saved to: {output_path}")
    return output_path

# -------------------------------------------------------------------
# CSV Summarizer Function
# -------------------------------------------------------------------
def analyze_csv(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Get column names and data types
    column_info = df.dtypes.reset_index()
    column_info.columns = ["Column Name", "Data Type"]

    # Get dataset size
    dataset_size = df.shape

    # Display results
    print("\nCSV Summary:")
    print("Columns and Data Types:")
    print(column_info.to_string(index=False))
    print(f"\nTotal Rows: {dataset_size[0]}")
    print(f"Total Columns: {dataset_size[1]}")

# -------------------------------------------------------------------
# Detect Non-Numeric Columns Function
# -------------------------------------------------------------------
def detect_non_numeric_columns(csv_file):
    """
    Reads csv_file and checks each column for non-numeric values.
    Returns a dictionary mapping column names to the number of entries that become NaN
    after converting to numeric (i.e. problematic entries).
    """
    df = pd.read_csv(csv_file, engine='python', on_bad_lines='skip', quoting=csv.QUOTE_NONE)
    non_numeric = {}
    for col in df.columns:
        try:
            converted = pd.to_numeric(df[col], errors='coerce')
            additional_nans = converted.isna() & ~df[col].isna()
            count = additional_nans.sum()
            if count > 0:
                non_numeric[col] = count
        except Exception as e:
            non_numeric[col] = str(e)
    return non_numeric

# -------------------------------------------------------------------
# Check for -999 Values Function
# -------------------------------------------------------------------
def check_minus999_values(csv_file):
    """
    Checks for occurrences of -999 in the dataset.
    Returns a dictionary mapping column names to the count of -999 values.
    """
    df = pd.read_csv(csv_file, engine='python', on_bad_lines='skip', quoting=csv.QUOTE_NONE)
    minus999_counts = {}
    for col in df.columns:
        try:
            numeric_series = pd.to_numeric(df[col], errors='coerce')
            count_minus999 = (numeric_series == -999).sum()
            if count_minus999 > 0:
                minus999_counts[col] = count_minus999
        except Exception as e:
            minus999_counts[col] = f"Error: {e}"
    return minus999_counts

# -------------------------------------------------------------------
# Main Processing: Clean, Summarize, and Check CSV Files
# -------------------------------------------------------------------
def main():
    folder_path = r"C:\Users\jmm267\Downloads\Binbin\Dataset\GEE_unseen_cleaned_time_series"  # Update if necessary.
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    print(f"Found {len(csv_files)} CSV files in {folder_path}")

    for csv_file in tqdm(csv_files, desc="Processing CSV files"):
        # Clean the CSV if not already cleaned
        if not csv_file.endswith("_cleaned.csv"):
            cleaned_path = clean_csv(csv_file)
        else:
            cleaned_path = csv_file

        print(f"\n--- Summary for: {cleaned_path} ---")
        analyze_csv(cleaned_path)

        print("\nChecking for non-numeric columns:")
        problems = detect_non_numeric_columns(cleaned_path)
        if problems:
            for col, count in problems.items():
                print(f"  {col}: {count} problematic entries")
        else:
            print("  All columns are numeric (or properly handled).")

        print("\nChecking for -999 values:")
        minus999_problems = check_minus999_values(cleaned_path)
        if minus999_problems:
            for col, count in minus999_problems.items():
                print(f"  {col}: {count} occurrences of -999")
        else:
            print("  No -999 values found.")
        print("-" * 50)

if __name__ == "__main__":
    main()


## (3) LSTM (long short term memory)

Run this code after cleaning the data. LSTM  is a type of RNN (recurrent neural network) that's best for time series data because it can "learn" the trends between times properly.

check for CUDA and tensorflow.

In [2]:
## CHECK FOR CUDA AND TENSORFLOW

print(f"TensorFlow Version: {tf.__version__}")
import torch
print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

TensorFlow Version: 2.18.0
Number of GPU:  1
GPU Name:  NVIDIA RTX 2000 Ada Generation
Using device: cuda


In [ ]:
## LSTM IMPLEMNETATION
## EXPORT WEIGHTS IN .PTH FORMAT "lstm_classifier.pth"
## RENAME GROUNDTRUTH CSV FILEPATH AT MAIN

import os
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == 'cuda':
    print("GPU Name:", torch.cuda.get_device_name())

##############################################################################
# 1) Dataset Class for 12-Month Sequences
##############################################################################
class GroundtruthTimeSeriesDataset(Dataset):
    """
    Reads a CSV with repeated rows over months for each (lat, lon),
    e.g. 12 rows for 12 months. We group them to form a single time-series
    sample: shape (12, number_of_features). The 'Class' label is taken
    from the first row's 'Class' (assuming each lat/lon is a single species).

    If a lat/lon has fewer than 12 months, we can pad with the last row.
    If it has more than 12, we can slice down to 12.

    The user can define which columns are the 'feature_cols'.
    We'll store 'label_encoder' for Class strings -> integers.
    """

    def __init__(self, csv_path, feature_cols=None, desired_length=12):
        print(f"Reading CSV file: {csv_path}")
        df = pd.read_csv(csv_path, parse_dates=['image_date'])

        if feature_cols is None:
            # You indicated at least 20 features: AOT..WVP, EVI, NDVI, NDWI, etc.
            # Adjust to exactly the ones you want. Example below:
            self.feature_cols = [
                "AOT","B1","B11","B12","B2","B3","B4","B5",
                "B6","B7","B8","B8A","B9","EVI","NDVI","NDWI",
                "TCI_B","TCI_G","TCI_R","WVP"
            ]
        else:
            self.feature_cols = feature_cols

        # We'll group by (latitude, longitude). If you prefer rounding, do so here.
        # e.g. df['lat_lon_id'] = ...
        # But let's assume the lat/lon values are consistent for each patch.
        df["lat_lon_id"] = df["latitude"].astype(str) + "_" + df["longitude"].astype(str)

        # Encode "Class" if present
        if "Class" not in df.columns:
            raise ValueError("CSV must have 'Class' column for supervised training.")

        # Build label encoder
        self.label_encoder = LabelEncoder()
        class_strs = df["Class"].unique()
        self.label_encoder.fit(class_strs)

        self.desired_length = desired_length
        self.samples = []  # each is a (time_series_tensor, label_int)

        # group by lat_lon_id
        grouped = df.groupby("lat_lon_id", sort=False)
        for group_id, group_data in grouped:
            group_data = group_data.sort_values("image_date")
            arr = group_data[self.feature_cols].to_numpy(dtype=float)  # shape (T, D)
            T, D = arr.shape

            if T < desired_length:
                # pad with last row
                pad = np.tile(arr[-1:], (desired_length - T, 1))
                arr = np.vstack([arr, pad])
            elif T > desired_length:
                arr = arr[:desired_length]

            # Convert to torch tensor
            seq_tensor = torch.tensor(arr, dtype=torch.float)  # shape (12, #features)
            # Label: from the first row's 'Class'
            class_str = group_data.iloc[0]["Class"]
            label_id = self.label_encoder.transform([class_str])[0]

            self.samples.append((seq_tensor, label_id))

        print(f"Constructed {len(self.samples)} time-series samples from {csv_path}.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        seq, label = self.samples[idx]
        return seq, label


##############################################################################
# 2) Neural Network Model: LSTM (or GRU) to handle 12×D sequences
##############################################################################
class LSTMClassifier(nn.Module):
    """
    An LSTM-based classifier that:
      - Reads sequences of shape (T=12, D=20+).
      - Optionally: you can add multiple LSTM layers, dropout, etc.
      - Then we take the final hidden state or last time step for classification.
    """

    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, drop_prob=0.2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # You can also use nn.GRU(...) if you prefer a GRU
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=drop_prob,
            bidirectional=False
        )
        # final classification layer
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # x: shape (B, T, D)
        # Initialize hidden + cell states:
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)

        # LSTM forward
        out, (hn, cn) = self.lstm(x, (h0, c0))
        # out shape: (B, T, hidden_dim)
        # hn shape: (num_layers, B, hidden_dim) => final hidden state for last time step

        # We'll take the last time step's output
        # or we can just use hn[-1], which is the last layer's hidden state
        last_out = out[:, -1, :]  # shape (B, hidden_dim)
        logits = self.fc(last_out)  # shape (B, num_classes)
        return logits


##############################################################################
# 3) Main Training Pipeline
##############################################################################
def train_lstm_classifier(
    csv_path,
    epochs=100,
    batch_size=32,
    hidden_dim=128,
    num_layers=1,
    drop_prob=0.2,
    save_path="lstm_classifier.pth"
):
    # 1) Create Dataset
    dataset = GroundtruthTimeSeriesDataset(csv_path)

    # 2) Train/Val Split
    ds_len = len(dataset)
    train_size = int(0.8 * ds_len)
    val_size = ds_len - train_size
    train_ds, val_ds = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    num_classes = len(dataset.label_encoder.classes_)
    input_dim = len(dataset.feature_cols)

    print(f"Dataset has {ds_len} time-series samples, {num_classes} distinct classes.")
    print(f"Using LSTM with input_dim={input_dim}, hidden_dim={hidden_dim}, num_classes={num_classes}.")
    print(f"Training for {epochs} epochs...")

    # 3) Build Model
    model = LSTMClassifier(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=num_classes,
        drop_prob=drop_prob
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # 4) Training Loop
    for epoch in range(1, epochs+1):
        # ---- train ----
        model.train()
        total_loss = 0.0
        correct, total = 0, 0
        for X, y in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs} (train)", leave=False):
            X = X.to(device)            # shape (B, 12, input_dim)
            y = y.to(device)            # shape (B,)
            optimizer.zero_grad()
            logits = model(X)           # shape (B, num_classes)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * X.size(0)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == y).sum().item()
            total += X.size(0)
        train_acc = correct / total
        avg_loss = total_loss / total

        # ---- validate ----
        model.eval()
        val_loss = 0.0
        val_correct, val_total = 0, 0
        all_val_preds = []
        all_val_labels = []
        with torch.no_grad():
            for Xv, yv in tqdm(val_loader, desc=f"Epoch {epoch}/{epochs} (val)", leave=False):
                Xv = Xv.to(device)
                yv = yv.to(device)
                logits_v = model(Xv)
                loss_v = criterion(logits_v, yv)
                val_loss += loss_v.item() * Xv.size(0)
                preds_v = torch.argmax(logits_v, dim=1)
                val_correct += (preds_v == yv).sum().item()
                val_total += Xv.size(0)
                all_val_preds.extend(preds_v.cpu().numpy())
                all_val_labels.extend(yv.cpu().numpy())

        val_acc = val_correct / val_total
        avg_val_loss = val_loss / val_total
        print(f"[Epoch {epoch}/{epochs}] Train Loss={avg_loss:.4f}, Train Acc={train_acc:.2f}, "
              f"Val Loss={avg_val_loss:.4f}, Val Acc={val_acc:.2f}")

    # 5) Save the model
    torch.save(model.state_dict(), save_path)
    print(f"\nModel saved to {save_path}.")

    # 6) Final Confusion Matrix on val set
    cm = confusion_matrix(all_val_labels, all_val_preds)
    print("\nConfusion Matrix (on validation set):")
    print(cm)

    # Classification Report
    print("\nClassification Report:")
    # Convert numeric labels back to strings
    inv_class_map = {i: c for i, c in enumerate(dataset.label_encoder.classes_)}
    all_val_preds_str = [inv_class_map[p] for p in all_val_preds]
    all_val_labels_str = [inv_class_map[l] for l in all_val_labels]
    print(classification_report(all_val_labels_str, all_val_preds_str))

    return model, dataset


##############################################################################
# 4) Example Usage
##############################################################################
if __name__ == "__main__":
    groundtruth_csv = r"C:\Users\jmm267\Downloads\Binbin\Dataset\groundtruth_raw\time_series_cleaned\groundtruth_cleaned_final.csv"
    model, ds = train_lstm_classifier(
        csv_path=groundtruth_csv,
        epochs=100,
        batch_size=32,
        hidden_dim=128,
        num_layers=2,
        drop_prob=0.2,
        save_path="lstm_classifier_model.pth"
    )


###  After training 100 epochs on the groundtruth file, we can access the .pth file and its weights to perform classification on unseen data.

In [ ]:
## Classifier code starts here ##
import os
import glob
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ---------------------------------------------------------------------
# 0. Check CUDA / GPU availability
# ---------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("=== Checking for GPU (CUDA) availability ===")
if device.type == 'cuda':
    print(f"GPU is available! Using device: {device}")
    print(f"GPU Name: {torch.cuda.get_device_name()}")
else:
    print("No GPU found. Running on CPU.")

# ---------------------------------------------------------------------
# 1. LSTM Model Architecture
# ---------------------------------------------------------------------
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, drop_prob=0.2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=drop_prob,
            bidirectional=False
        )
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        batch_size = x.size(0)
        # Initialize hidden/cell
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))
        # last time-step output
        last_out = out[:, -1, :]  # shape (B, hidden_dim)
        logits = self.fc(last_out)
        return logits

# ---------------------------------------------------------------------
# 2. BlindPatchDataset for unlabeled CSV
# ---------------------------------------------------------------------
class BlindPatchDataset(Dataset):
    """
    For a single CSV with columns:
      [AOT,B1,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,B9,EVI,NDVI,NDWI,
       TCI_B,TCI_G,TCI_R,WVP, image_date,longitude,latitude]
    We group by lat/lon, sort by image_date, ensure T=12 (pad or truncate).
    Returns (seq_tensor, (lat, lon)) for each pixel/time-series.
    """

    def __init__(self, csv_path, feature_cols=None, desired_length=12):
        print(f"\n--- Reading CSV file: {csv_path}")
        self.df = pd.read_csv(csv_path, parse_dates=["image_date"])
        print(f"[DEBUG] Shape of read DataFrame: {self.df.shape}")

        if feature_cols is None:
            self.feature_cols = [
                "AOT","B1","B11","B12","B2","B3","B4","B5",
                "B6","B7","B8","B8A","B9","EVI","NDVI","NDWI",
                "TCI_B","TCI_G","TCI_R","WVP"
            ]
        else:
            self.feature_cols = feature_cols

        # Combine lat/lon into a group id
        self.df["lat_lon_id"] = (
            self.df["latitude"].round(6).astype(str)
            + "_"
            + self.df["longitude"].round(6).astype(str)
        )

        self.desired_length = desired_length
        self.samples = []
        self.latlons = []

        grouped = self.df.groupby("lat_lon_id", sort=False)
        for group_id, gdata in grouped:
            gdata_sorted = gdata.sort_values("image_date")
            arr = gdata_sorted[self.feature_cols].to_numpy(dtype=float)  # shape (T, D)
            T, D = arr.shape

            if T < desired_length:
                # pad with last row
                pad = np.tile(arr[-1:], (desired_length - T, 1))
                arr = np.vstack([arr, pad])
            elif T > desired_length:
                arr = arr[:desired_length]

            seq_tensor = torch.tensor(arr, dtype=torch.float)
            lat0 = gdata_sorted.iloc[0]["latitude"]
            lon0 = gdata_sorted.iloc[0]["longitude"]
            self.samples.append(seq_tensor)
            self.latlons.append((lat0, lon0))

        print(f"[DEBUG] Constructed {len(self.samples)} time-series samples.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.latlons[idx]

# ---------------------------------------------------------------------
# 2.5 Custom Collate Function
# ---------------------------------------------------------------------
def collate_blind_patches(batch):
    """
    Expects a list of (seq_tensor, (lat, lon)) pairs.
    We'll stack all seq_tensors into (B, T, D),
    and keep coords as a list of (lat, lon).
    """
    seqs = []
    coords = []
    for (seq_tensor, (lat, lon)) in batch:
        seqs.append(seq_tensor)
        coords.append((lat, lon))

    # Stack the sequence tensors along the batch dimension
    seq_batch = torch.stack(seqs, dim=0)  # shape (B, 12, feature_dim)

    return seq_batch, coords

# ---------------------------------------------------------------------
# 3. Inference Function
# ---------------------------------------------------------------------
def infer_on_blind_data(
    folder_path,                   # path to raw_cleaned folder
    model_path="lstm_classifier_model.pth",
    output_folder="inference_results",
    batch_size=32,
    hidden_dim=128,
    num_layers=2,
    drop_prob=0.2,
    feature_cols=None
):
    """
    - folder_path: Directory that contains *cleaned.csv files
    - model_path: Path to .pth model weights
    - output_folder: Where we store the results
    - batch_size, hidden_dim, num_layers, drop_prob: must match your training config
    - feature_cols: same features used in training
    """
    print("\n=== Starting Inference on Blind Data ===")
    csv_files = glob.glob(os.path.join(folder_path, "*cleaned.csv"))
    print(f"Found {len(csv_files)} files in {folder_path} that match '*cleaned.csv'")

    print(f"Model path: {model_path}")
    print(f"Output folder: {output_folder}")
    print("Feature cols:", feature_cols)
    print("--------------------------------------------------")

    # Suppose you have 5 classes from training
    num_classes = 5
    input_dim = len(feature_cols) if feature_cols else 20

    # Rebuild the same LSTM
    print(f"[INFO] Building LSTM model: input_dim={input_dim}, hidden_dim={hidden_dim}, num_classes={num_classes}")
    model = LSTMClassifier(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=num_classes,
        drop_prob=drop_prob
    )
    print(f"[INFO] Loading state_dict from: {model_path}")
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    # Label mapping from training
    label_mapping = ["deci_broad", "ever_coni", "larch_CN", "larch_JP", "shrubland"]

    # Ensure output folder
    os.makedirs(output_folder, exist_ok=True)

    for csv_path in tqdm(csv_files, desc="CSV Files", unit="file"):
        print(f"\n--- Inference on file: {csv_path}")
        try:
            ds = BlindPatchDataset(csv_path, feature_cols=feature_cols)
            # Pass our custom collate function:
            dl = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate_blind_patches)

            print(f"[DEBUG] Dataset has {len(ds)} samples. Beginning inference...")

            all_preds = []
            all_coords = []

            with torch.no_grad():
                for seq_batch, coords_batch in tqdm(dl, desc="Predicting Batches", leave=False):
                    # seq_batch: shape (B, 12, input_dim)
                    # coords_batch: list of length B, each is (lat, lon)
                    seq_batch = seq_batch.to(device)
                    logits = model(seq_batch)
                    preds = torch.argmax(logits, dim=1).cpu().numpy()
                    all_preds.extend(preds)
                    all_coords.extend(coords_batch)

            if len(all_preds) != len(all_coords):
                raise ValueError(f"[ERROR] length mismatch: preds={len(all_preds)}, coords={len(all_coords)}")

            pred_classes = [label_mapping[p] for p in all_preds]

            # Build results DataFrame
            print(f"[DEBUG] Building DataFrame for {len(pred_classes)} predictions...")
            results_df = pd.DataFrame({
                "latitude": [c[0] for c in all_coords],
                "longitude": [c[1] for c in all_coords],
                "predicted_class": pred_classes
            })
            print(results_df.head(5))

            base_name = os.path.splitext(os.path.basename(csv_path))[0]
            out_csv = os.path.join(output_folder, base_name + "_with_predictions.csv")
            results_df.to_csv(out_csv, index=False)
            print(f"Results saved to: {out_csv}")

        except Exception as e:
            print(f"[ERROR] Failed on file {csv_path}\nReason: {e}")

# ---------------------------------------------------------------------
# 4. Example usage
# ---------------------------------------------------------------------
if __name__ == "__main__":
    folder_path = r"C:\Users\jmm267\Downloads\Binbin\Dataset\raw_cleaned"
    ## runs classification on all csv files in this directory
    features_used = [
        "AOT","B1","B11","B12","B2","B3","B4","B5",
        "B6","B7","B8","B8A","B9","EVI","NDVI","NDWI",
        "TCI_B","TCI_G","TCI_R","WVP"
    ]
    ## indicate which features to use

    ## call the main function here
    infer_on_blind_data(
        folder_path=folder_path,
        model_path="lstm_classifier_model.pth",
        ##update the trained model path
        output_folder="larchCN_inference_results",
        ##output the classified data into this folder
        batch_size=32,
        hidden_dim=128,
        num_layers=2,
        drop_prob=0.2,
        feature_cols=features_used
    )
